In [1]:
%%time
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

CPU times: user 812 ms, sys: 274 ms, total: 1.09 s
Wall time: 913 ms


In [2]:
%%time
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE,RandomOverSampler
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from itertools import combinations 
from mlxtend.classifier import StackingClassifier
from sklearn import model_selection

Using TensorFlow backend.
/home/sf/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sf/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sf/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sf/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtype

CPU times: user 1.46 s, sys: 371 ms, total: 1.83 s
Wall time: 16.9 s


In [7]:
df=pd.read_csv('60s_window_wrist_chest.csv',index_col=0)
df['label'].value_counts()[0]

312

In [12]:
print(df['label'].value_counts()[1]/(len(list(df['label']))))

0.2229299363057325


In [8]:
df=pd.read_csv('60s_window_wrist_chest.csv',index_col=0)
features=df.columns.tolist()

removed = ['label']
for rem in removed:
    features.remove(rem)
features_with_sub=[]
features_with_sub[:]=features
removed = ['subject']
for rem in removed:
    features.remove(rem)

feature=features
print(len(feature))
len(features_with_sub)

sm = SMOTE(random_state=2)
X, y= sm.fit_sample(df[features_with_sub], df['label'])
df_new=pd.concat([pd.DataFrame(X,columns=features_with_sub),pd.DataFrame(y,columns=['label'])],axis=1)


for i in range (len(list(df_new['subject']))):
    df_new['subject'][i] = min([2,3,4,5,6,7,8,9,10,11,13,14,15,16,17], key=lambda x:abs(x-df_new['subject'][i]))
df_new['subject']=df_new['subject'].astype(int)


p_d=pd.read_csv('personal_detail.csv',index_col=0)


df_new_1=df_new.merge(p_d,on='subject')


features=df_new_1.columns.tolist()
features

removed = ['label']
for rem in removed:
    features.remove(rem)
features_with_sub=[]
features_with_sub[:]=features
removed = ['subject']
for rem in removed:
    features.remove(rem)

feature=features
print(len(feature))
len(features_with_sub)

70


/home/sf/.local/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


79


80

In [9]:
train=df_new_1[df_new_1['subject']<=9]
test=df_new_1[df_new_1['subject']>9]

In [10]:
et = ExtraTreesClassifier(n_estimators=100,n_jobs=10,random_state=56 ,)
scaler = Normalizer()
scaled_data_train = scaler.fit_transform(train[feature])
scaled_data_test = scaler.transform(test[feature])
et.fit(scaled_data_train,train['label'])
y_pred=et.predict(scaled_data_test)
print(classification_report(test['label'],y_pred))

              precision    recall  f1-score   support

           0       0.77      0.62      0.69       147
           1       0.90      0.92      0.91       161
           2       0.65      0.79      0.71       146
           3       0.76      0.74      0.75       149

    accuracy                           0.77       603
   macro avg       0.77      0.77      0.77       603
weighted avg       0.77      0.77      0.77       603



In [30]:
#activation : {‘identity’, ‘logistic’, ‘tanh’, ‘relu’}
#solver : {‘lbfgs’, ‘sgd’, ‘adam’}
model = MLPClassifier(verbose=2,max_iter=1000,hidden_layer_sizes=(500,200),activation='identity',solver='lbfgs')
model.fit(scaled_data_train,train['label'])

MLPClassifier(activation='identity', alpha=0.0001, batch_size='auto',
              beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(500, 200), learning_rate='constant',
              learning_rate_init=0.001, max_iter=1000, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=2, warm_start=False)

In [28]:
y_pred=model.predict(scaled_data_test)
print(classification_report(test['label'],y_pred))

              precision    recall  f1-score   support

           0       0.57      0.44      0.49       147
           1       0.68      0.76      0.72       161
           2       0.39      0.42      0.41       146
           3       0.64      0.64      0.64       149

    accuracy                           0.57       603
   macro avg       0.57      0.57      0.57       603
weighted avg       0.57      0.57      0.57       603

